In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using RigidBodyDynamics
using RigidBodySim
using MeshCatMechanisms
using StrandbeestRobot

using StaticArrays: SVector
using LinearAlgebra: normalize
using RigidBodyDynamics.Contact: HalfSpace3D
using GeometryTypes: HyperRectangle, Vec
using CoordinateTransformations: LinearMap
using Colors: RGB
using Rotations: RotY, Quat, rotation_between
using MeshCat: setobject!, settransform!, MeshPhongMaterial

In [ ]:
mechanism = StrandbeestRobot.mechanism(add_flat_ground=false)
ground_normal = normalize(SVector(1., 0, 5))
add_environment_primitive!(mechanism, 
    HalfSpace3D(Point3D(root_frame(mechanism), 0., 0, 0),
        FreeVector3D(root_frame(mechanism), ground_normal)))

gui = GUI(mechanism, URDFVisuals(StrandbeestRobot.urdfpath()));

In [ ]:
#NBSKIP
open(gui);

In [ ]:
state = MechanismState(mechanism)
StrandbeestRobot.solve_initial_configuration!(state)

In [ ]:
floatingjoint = findjoint(mechanism, "floating_base")
rot = Quat(rotation_between(SVector(0., 0, 1), ground_normal))
tf = Transform3D(frame_after(floatingjoint), frame_before(floatingjoint), rot, SVector(0, 0, 0.9))
set_configuration!(state, floatingjoint, tf)

setobject!(gui.visualizer.visualizer[:ground], HyperRectangle(Vec(-50, -2, -0.01), Vec(100, 4, 0.02)), 
MeshPhongMaterial(color=RGB(0.5, 0.5, 0.5)))
settransform!(gui.visualizer.visualizer[:ground], LinearMap(rot))

copyto!(gui.visualizer, state)

In [ ]:
# Simulate the robot
damping = function (τ, t, state)
    τ .= -0.1 .* velocity(state)
end
dynamics = Dynamics(mechanism, damping)
problem = ODEProblem(dynamics, state, (0, 5.0))
@time solution = solve(problem, Tsit5(), abs_tol=1e-6, dt=1e-6)
setanimation!(gui.visualizer, solution)